In [ ]:
## this script creates a new noisy dataset with null values and outliers included
## the new dataset is created and was given the name out.csv
## in order to fast forward the process a copy of the first file that we created was uploaded to https://github.com/gkolovos/kaggleDATAset
## the out.csv file in that path is the file that we used in our work in order to perform the denoising model
## please note that this script creates random noise in the whole dataset so keep in mind that if re-run the script a new out.csv file will be created
## for our convenience we uploaded the file we used in our work on github


#install pyspark
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import seaborn as sns

In [ ]:
spark = SparkSession.builder.appName('Heart_Disease_pySpark').getOrCreate()
spark

In [ ]:
! git clone https://github.com/gkolovos/kaggleDATAset

fatal: destination path 'kaggleDATAset' already exists and is not an empty directory.


In [ ]:
ls kaggleDATAset

heart.csv  noisyfile2.csv  out.csv


In [ ]:
#create spark dataframe
df = spark.read.csv("/content/kaggleDATAset/heart.csv", header=True, inferSchema=True )


In [ ]:
df.show()

+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| 40|  M|          ATA|      140|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|
| 49|  F|          NAP|      160|        180|        0|    Normal|  156|             N|    1.0|    Flat|           1|
| 37|  M|          ATA|      130|        283|        0|        ST|   98|             N|    0.0|      Up|           0|
| 48|  F|          ASY|      138|        214|        0|    Normal|  108|             Y|    1.5|    Flat|           1|
| 54|  M|          NAP|      150|        195|        0|    Normal|  122|             N|    0.0|      Up|           0|
| 39|  M|          NAP|      120|        339|        0| 

In [ ]:
##### metatrepw oles tis katigorimatikes (px F h' M, TA h' ASY klp klp) times se arithmous gia na eisagw thoryvo

In [ ]:
#varitita tou chest pain type
def ChestPainType_importance(col):
    if col == 'NAP':
        return 0.0
    elif col == 'ASY':
        return 0.5
    elif col == 'ATA':
        return 0.75
    else:
         return 1.0

In [ ]:
ChestPainType_udf = udf(lambda val: ChestPainType_importance(val), DoubleType())
df_after_edit_ChestPainType = df.withColumn('ChestPainType',ChestPainType_udf('ChestPainType'))

In [ ]:
df_after_edit_ChestPainType.show(10)


+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| 40|  M|         0.75|      140|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|
| 49|  F|          0.0|      160|        180|        0|    Normal|  156|             N|    1.0|    Flat|           1|
| 37|  M|         0.75|      130|        283|        0|        ST|   98|             N|    0.0|      Up|           0|
| 48|  F|          0.5|      138|        214|        0|    Normal|  108|             Y|    1.5|    Flat|           1|
| 54|  M|          0.0|      150|        195|        0|    Normal|  122|             N|    0.0|      Up|           0|
| 39|  M|          0.0|      120|        339|        0| 

In [ ]:
def ExerciseAngina_val(col):
    if col == 'Y':
        return 1
    else:
        return 0

In [ ]:
ExerciseAngina_udf = udf(lambda val: ExerciseAngina_val(val), IntegerType())
df_after_edit_ExerciseAngina = df_after_edit_ChestPainType.withColumn('ExerciseAngina', ExerciseAngina_udf('ExerciseAngina'))


In [ ]:
def sex_val(col):
    if col == 'F':
        return 0
    else:
        return 1

In [ ]:
sex_udf = udf(lambda val: sex_val(val), IntegerType())
df_after_edit_sex = df_after_edit_ExerciseAngina.withColumn('sex', sex_udf('sex'))

In [ ]:
final_df = df_after_edit_sex

In [ ]:
def RestingECG_val(col):
    if col == 'Normal':
        return 0.0
    elif col == 'ST':
        return 0.75
    else:
        return 1.0

In [ ]:
RestingECG_udf = udf(lambda val: RestingECG_val(val), DoubleType())
df_after_edit_RestingECG = final_df.withColumn('RestingECG', RestingECG_udf('RestingECG'))

In [ ]:
df_after_edit_RestingECG.show()

+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| 40|  1|         0.75|      140|        289|        0|       0.0|  172|             0|    0.0|      Up|           0|
| 49|  0|          0.0|      160|        180|        0|       0.0|  156|             0|    1.0|    Flat|           1|
| 37|  1|         0.75|      130|        283|        0|      0.75|   98|             0|    0.0|      Up|           0|
| 48|  0|          0.5|      138|        214|        0|       0.0|  108|             1|    1.5|    Flat|           1|
| 54|  1|          0.0|      150|        195|        0|       0.0|  122|             0|    0.0|      Up|           0|
| 39|  1|          0.0|      120|        339|        0| 

In [ ]:
def ST_Slope_val(col):
    if col == 'Flat':
        return 0.0
    elif col == 'Up':
        return 0.75
    else:
        return 1.0

In [ ]:
ST_Slope_udf = udf(lambda val: ST_Slope_val(val), DoubleType())
df_teliko = df_after_edit_RestingECG.withColumn('ST_Slope', ST_Slope_udf('ST_Slope'))

In [ ]:
df_teliko.show()

+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| 40|  1|         0.75|      140|        289|        0|       0.0|  172|             0|    0.0|    0.75|           0|
| 49|  0|          0.0|      160|        180|        0|       0.0|  156|             0|    1.0|     0.0|           1|
| 37|  1|         0.75|      130|        283|        0|      0.75|   98|             0|    0.0|    0.75|           0|
| 48|  0|          0.5|      138|        214|        0|       0.0|  108|             1|    1.5|     0.0|           1|
| 54|  1|          0.0|      150|        195|        0|       0.0|  122|             0|    0.0|    0.75|           0|
| 39|  1|          0.0|      120|        339|        0| 

In [ ]:
df_teliko.groupBy("sex").count().show()


+---+-----+
|sex|count|
+---+-----+
|  1|  725|
|  0|  193|
+---+-----+



In [ ]:
df_teliko.groupBy("Oldpeak").count().show()


+-------+-----+
|Oldpeak|count|
+-------+-----+
|    2.4|    4|
|    0.0|  368|
|    3.5|    2|
|    0.2|   22|
|   -0.1|    2|
|   -2.0|    1|
|    2.9|    1|
|    3.7|    1|
|   -1.0|    2|
|    1.4|   18|
|    1.7|    6|
|    0.7|    7|
|    2.3|    2|
|   -0.9|    1|
|    0.1|   14|
|    3.4|    3|
|    2.5|   16|
|    1.0|   86|
|   -1.1|    1|
|    0.6|   14|
+-------+-----+
only showing top 20 rows



In [ ]:
df_teliko.groupBy("FastingBS").count().show()


+---------+-----+
|FastingBS|count|
+---------+-----+
|        1|  214|
|        0|  704|
+---------+-----+



In [ ]:
df_teliko.groupBy("ExerciseAngina").count().show()


+--------------+-----+
|ExerciseAngina|count|
+--------------+-----+
|             1|  371|
|             0|  547|
+--------------+-----+



In [ ]:
df_teliko.groupBy("HeartDisease").count().show()


+------------+-----+
|HeartDisease|count|
+------------+-----+
|           1|  508|
|           0|  410|
+------------+-----+



In [ ]:
### metatrepw to spark df se pandas df gia na kanw tis metatropes 

pandasDF = df_teliko.toPandas()

In [ ]:
print(pandasDF)

     Age  sex  ChestPainType  RestingBP  Cholesterol  FastingBS  RestingECG  \
0     40    1           0.75        140          289          0        0.00   
1     49    0           0.00        160          180          0        0.00   
2     37    1           0.75        130          283          0        0.75   
3     48    0           0.50        138          214          0        0.00   
4     54    1           0.00        150          195          0        0.00   
..   ...  ...            ...        ...          ...        ...         ...   
913   45    1           1.00        110          264          0        0.00   
914   68    1           0.50        144          193          1        0.00   
915   57    1           0.50        130          131          0        0.00   
916   57    0           0.75        130          236          0        1.00   
917   38    1           0.00        138          175          0        0.00   

     MaxHR  ExerciseAngina  Oldpeak  ST_Slope  Hear

In [ ]:
###############################################################  Creating noise ###########################################################################

# mu is the mean of the normal distribution we are choosing from
# std is the standard deviation

mu=0.0
std = 0.1
def gaussian_noise(pandasDF,mu,std):
    noise = np.random.normal(mu, std, size = pandasDF.shape)
    x_noisy = pandasDF + noise
    return x_noisy 

In [ ]:
###############################################################  adding noise ###########################################################################
import numpy as np

x_noisy = pandasDF + np.random.normal(mu, std, size = pandasDF.shape)

In [ ]:
print(x_noisy)

           Age       sex  ChestPainType   RestingBP  Cholesterol  FastingBS  \
0    40.075810  0.896255       0.585498  140.028725   289.047493  -0.110816   
1    49.026583 -0.155517      -0.033728  159.951914   179.976347   0.066833   
2    37.027058  1.213124       0.689128  129.886267   283.062885   0.259855   
3    48.026006 -0.023906       0.414232  137.890466   214.003089   0.025362   
4    54.104653  0.999833      -0.189660  149.997806   194.841817  -0.115982   
..         ...       ...            ...         ...          ...        ...   
913  45.016117  1.074887       0.984382  109.982578   264.061031  -0.198210   
914  68.027553  0.894708       0.312872  144.178860   193.091814   1.121330   
915  56.815044  1.079318       0.428938  129.978099   131.002007   0.063410   
916  57.221658 -0.026099       0.660601  129.952755   235.816610  -0.108806   
917  38.127729  0.999436       0.109322  137.992164   174.968208   0.101983   

     RestingECG       MaxHR  ExerciseAngina   Oldpe

In [ ]:
import pandas as pd
import numpy as np

x_noisy['HeartDisease'] = np.where(x_noisy['HeartDisease'] > 0.7 ,1,0) 
#pd.cut(x_noisy['HeartDisease'], bins=[0, 0.45, 0.55, 10], include_lowest=True, labels=['0', 'NaN', '1'])

In [ ]:
x_noisy.groupby(['HeartDisease']).size()


HeartDisease
0    412
1    506
dtype: int64

In [ ]:
print(x_noisy)

           Age       sex  ChestPainType   RestingBP  Cholesterol  FastingBS  \
0    40.075810  0.896255       0.585498  140.028725   289.047493  -0.110816   
1    49.026583 -0.155517      -0.033728  159.951914   179.976347   0.066833   
2    37.027058  1.213124       0.689128  129.886267   283.062885   0.259855   
3    48.026006 -0.023906       0.414232  137.890466   214.003089   0.025362   
4    54.104653  0.999833      -0.189660  149.997806   194.841817  -0.115982   
..         ...       ...            ...         ...          ...        ...   
913  45.016117  1.074887       0.984382  109.982578   264.061031  -0.198210   
914  68.027553  0.894708       0.312872  144.178860   193.091814   1.121330   
915  56.815044  1.079318       0.428938  129.978099   131.002007   0.063410   
916  57.221658 -0.026099       0.660601  129.952755   235.816610  -0.108806   
917  38.127729  0.999436       0.109322  137.992164   174.968208   0.101983   

     RestingECG       MaxHR  ExerciseAngina   Oldpe

In [ ]:
# times  st age
x_noisy.groupby(['Age']).size()


Age
28.095811    1
28.854160    1
29.054906    1
29.133851    1
29.924839    1
            ..
75.002240    1
75.967075    1
76.177449    1
77.032777    1
77.135896    1
Length: 918, dtype: int64

In [ ]:
## categorize sex values below or above 0.5
 
x_noisy['sex'] = np.where(x_noisy['sex'] >= 0.5 ,1,0) 

In [ ]:
x_noisy.groupby(['sex']).size()


sex
0    193
1    725
dtype: int64

In [ ]:
#### O arithos tn sex=1 kai sex=0 dn allaxe opote tha allaxw liges random times apo 0 se 1 
### allazw tyxaia to 5% ton 1 se 0

v = x_noisy.sex.values == 1
x_noisy.loc[v, 'sex'] = np.random.choice((0, 1), v.sum(), p=(.05, .95))

In [ ]:
x_noisy.groupby(['sex']).size()

sex
0    224
1    694
dtype: int64

In [ ]:
print(x_noisy)

           Age  sex  ChestPainType   RestingBP  Cholesterol  FastingBS  \
0    40.075810    1       0.585498  140.028725   289.047493  -0.110816   
1    49.026583    0      -0.033728  159.951914   179.976347   0.066833   
2    37.027058    1       0.689128  129.886267   283.062885   0.259855   
3    48.026006    0       0.414232  137.890466   214.003089   0.025362   
4    54.104653    1      -0.189660  149.997806   194.841817  -0.115982   
..         ...  ...            ...         ...          ...        ...   
913  45.016117    1       0.984382  109.982578   264.061031  -0.198210   
914  68.027553    1       0.312872  144.178860   193.091814   1.121330   
915  56.815044    1       0.428938  129.978099   131.002007   0.063410   
916  57.221658    0       0.660601  129.952755   235.816610  -0.108806   
917  38.127729    0       0.109322  137.992164   174.968208   0.101983   

     RestingECG       MaxHR  ExerciseAngina   Oldpeak  ST_Slope  HeartDisease  
0     -0.102702  171.997752    

In [ ]:
# replace negative numbers with NaN
#x_noisy[x_noisy < 0] = "NaN"

## skeytika na krathsw orisenes arnhtikes times gia na yparxoyn outliners st dataframe 
## tha kanw random nan values sth synexeia

In [ ]:
#x_noisy[x_noisy < 0] = 0


In [ ]:
### FastingBS

x_noisy.groupby(['FastingBS']).size()

### exei times apo -0.32    ews 1.26
### mporw eykola na kanw osa einai > 0.5 na ta kanw 1 kai ta ypoloia 0
### skeytika na afisw merikes akrees times opws px ta < -0.30 kai > 1.2 gia na dhmiourgithoun outliners st dataset kai na na ta epexergastw meta 

FastingBS
-0.278739    1
-0.272865    1
-0.240999    1
-0.238892    1
-0.236990    1
            ..
 1.213444    1
 1.234011    1
 1.288773    1
 1.314724    1
 1.331417    1
Length: 918, dtype: int64

In [ ]:
#x_noisy['FastingBS'] = pd.cut(x_noisy['FastingBS'], bins=[-0.326525 , -0.25, 0.65, 0.85, 1.264831], include_lowest=True, labels=['0', '0.5', '0.75','1'])
x_noisy['FastingBS'] = pd.cut(x_noisy['FastingBS'], bins=[-0.455938, -0.254442, -0.250807, -0.234021, -0.224653, -0.204652, 0.5, 1.190075, 1.203776, 1.220578, 1.242796, 1.259308, 1.495296], include_lowest=True, labels=['-0.326524', '-0.300144', '-0.287943', '-0.284272', '-0.279712', '0.0', '1.0', '1.213249', '1.215685', '1.226302', '1.228461', '1.264830'])

In [ ]:
x_noisy.groupby(['FastingBS']).size()


### yparxoun merikes times akyres apo tis klassikes 0 kai 1, dld outliners

FastingBS
-0.326524      2
-0.300144      0
-0.287943      3
-0.284272      1
-0.279712      4
0.0          694
1.0          207
1.213249       0
1.215685       3
1.226302       1
1.228461       0
1.264830       3
dtype: int64

In [ ]:
### ExerciseAngina
x_noisy.groupby(['ExerciseAngina']).size()

ExerciseAngina
-0.293238    1
-0.290557    1
-0.252755    1
-0.247608    1
-0.246634    1
            ..
 1.257193    1
 1.260405    1
 1.263740    1
 1.268789    1
 1.322482    1
Length: 918, dtype: int64

In [ ]:
x_noisy['ExerciseAngina'] = pd.cut(x_noisy['ExerciseAngina'], bins=[-0.455938, -0.354442, -0.300807, -0.254021, -0.224653, -0.204652, 0.5, 1.200075, 1.203776, 1.220578, 1.242796, 1.259308, 1.495296], include_lowest=True, labels=['-0.316524', '-0.290144', '-0.287943', '-0.284272', '-0.279712', '0.0', '1.0', '1.213249', '1.215685', '1.226302', '1.228461', '1.264830'])

In [ ]:
x_noisy.groupby(['ExerciseAngina']).size()


ExerciseAngina
-0.316524      0
-0.290144      0
-0.287943      2
-0.284272      4
-0.279712      3
0.0          538
1.0          360
1.213249       0
1.215685       4
1.226302       2
1.228461       1
1.264830       4
dtype: int64

In [ ]:
print(x_noisy)

           Age  sex  ChestPainType   RestingBP  Cholesterol FastingBS  \
0    40.075810    1       0.585498  140.028725   289.047493       0.0   
1    49.026583    0      -0.033728  159.951914   179.976347       0.0   
2    37.027058    1       0.689128  129.886267   283.062885       0.0   
3    48.026006    0       0.414232  137.890466   214.003089       0.0   
4    54.104653    1      -0.189660  149.997806   194.841817       0.0   
..         ...  ...            ...         ...          ...       ...   
913  45.016117    1       0.984382  109.982578   264.061031       0.0   
914  68.027553    1       0.312872  144.178860   193.091814       1.0   
915  56.815044    1       0.428938  129.978099   131.002007       0.0   
916  57.221658    0       0.660601  129.952755   235.816610       0.0   
917  38.127729    0       0.109322  137.992164   174.968208       0.0   

     RestingECG       MaxHR ExerciseAngina   Oldpeak  ST_Slope  HeartDisease  
0     -0.102702  171.997752            0.0 -

In [ ]:
# FastingBS: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
# ExerciseAngina exercise induced angina (1 = yes; 0 = no)

# den gnt na exw hlikia 43,25 opote ta kanw ola na exoun morfh 43                  
cols = ['Age']                  
x_noisy[cols] = x_noisy[cols].applymap(np.int64)

In [ ]:
print(x_noisy)

     Age  sex  ChestPainType   RestingBP  Cholesterol FastingBS  RestingECG  \
0     40    1       0.585498  140.028725   289.047493       0.0   -0.102702   
1     49    0      -0.033728  159.951914   179.976347       0.0    0.037417   
2     37    1       0.689128  129.886267   283.062885       0.0    0.803815   
3     48    0       0.414232  137.890466   214.003089       0.0    0.015360   
4     54    1      -0.189660  149.997806   194.841817       0.0    0.026305   
..   ...  ...            ...         ...          ...       ...         ...   
913   45    1       0.984382  109.982578   264.061031       0.0    0.019594   
914   68    1       0.312872  144.178860   193.091814       1.0    0.095896   
915   56    1       0.428938  129.978099   131.002007       0.0   -0.081912   
916   57    0       0.660601  129.952755   235.816610       0.0    0.831146   
917   38    0       0.109322  137.992164   174.968208       0.0    0.080039   

          MaxHR ExerciseAngina   Oldpeak  ST_Slope 

In [ ]:
#### ChestPainType
x_noisy.groupby(['ChestPainType']).size()

ChestPainType
-0.236947    1
-0.230686    1
-0.216268    1
-0.214286    1
-0.195760    1
            ..
 1.170313    1
 1.194120    1
 1.210446    1
 1.239141    1
 1.255514    1
Length: 918, dtype: int64

In [ ]:
x_noisy['ChestPainType'] = pd.cut(x_noisy['ChestPainType'], bins=[-0.45, -0.20, -0.19, 0.35, 0.65, 0.85, 1.15, 1.20, 1.40 ], include_lowest=True, labels=['-0.30', '-0.20','0', '0.5', '0.75','1', '1.19', '1.39'])

In [ ]:
x_noisy.groupby(['ChestPainType']).size()

ChestPainType
-0.30      4
-0.20      2
0        231
0.5      462
0.75     146
1         67
1.19       3
1.39       3
dtype: int64

In [ ]:
print(x_noisy)

     Age  sex ChestPainType   RestingBP  Cholesterol FastingBS  RestingECG  \
0     40    1           0.5  140.028725   289.047493       0.0   -0.102702   
1     49    0             0  159.951914   179.976347       0.0    0.037417   
2     37    1          0.75  129.886267   283.062885       0.0    0.803815   
3     48    0           0.5  137.890466   214.003089       0.0    0.015360   
4     54    1             0  149.997806   194.841817       0.0    0.026305   
..   ...  ...           ...         ...          ...       ...         ...   
913   45    1             1  109.982578   264.061031       0.0    0.019594   
914   68    1             0  144.178860   193.091814       1.0    0.095896   
915   56    1           0.5  129.978099   131.002007       0.0   -0.081912   
916   57    0          0.75  129.952755   235.816610       0.0    0.831146   
917   38    0             0  137.992164   174.968208       0.0    0.080039   

          MaxHR ExerciseAngina   Oldpeak  ST_Slope  HeartDiseas

In [ ]:
### RestingECG
x_noisy.groupby(['RestingECG']).size()

RestingECG
-0.295294    1
-0.273575    1
-0.251777    1
-0.251216    1
-0.243215    1
            ..
 1.184473    1
 1.198268    1
 1.244757    1
 1.247359    1
 1.267017    1
Length: 918, dtype: int64

In [ ]:
x_noisy['RestingECG'] = pd.cut(x_noisy['RestingECG'], bins=[-0.40,-0.30, 0.55, 0.85, 1.25, 1.50], include_lowest=True, labels=['-0.35','0', '0.75','1','1.4'])

In [ ]:
x_noisy.groupby(['RestingECG']).size()

RestingECG
-0.35      0
0        554
0.75     159
1        204
1.4        1
dtype: int64

In [ ]:
print(x_noisy)

     Age  sex ChestPainType   RestingBP  Cholesterol FastingBS RestingECG  \
0     40    1           0.5  140.028725   289.047493       0.0          0   
1     49    0             0  159.951914   179.976347       0.0          0   
2     37    1          0.75  129.886267   283.062885       0.0       0.75   
3     48    0           0.5  137.890466   214.003089       0.0          0   
4     54    1             0  149.997806   194.841817       0.0          0   
..   ...  ...           ...         ...          ...       ...        ...   
913   45    1             1  109.982578   264.061031       0.0          0   
914   68    1             0  144.178860   193.091814       1.0          0   
915   56    1           0.5  129.978099   131.002007       0.0          0   
916   57    0          0.75  129.952755   235.816610       0.0       0.75   
917   38    0             0  137.992164   174.968208       0.0          0   

          MaxHR ExerciseAngina   Oldpeak  ST_Slope  HeartDisease  
0    171

In [ ]:
### ST_Slope
x_noisy.groupby(['ST_Slope']).size()

ST_Slope
-0.367146    1
-0.281263    1
-0.262701    1
-0.262580    1
-0.236531    1
            ..
 1.160021    1
 1.161948    1
 1.167205    1
 1.171566    1
 1.210655    1
Length: 918, dtype: int64

In [ ]:
x_noisy['ST_Slope'] = pd.cut(x_noisy['ST_Slope'], bins=[-0.40, -0.25, 0.55, 0.85, 1.15, 1.29, 1.50], include_lowest=True, labels=['-40','0', '0.75','1','1.29','1.50'])


In [ ]:
x_noisy.groupby(['ST_Slope']).size()

ST_Slope
-40       4
0       465
0.75    324
1       118
1.29      7
1.50      0
dtype: int64

In [ ]:
#### oi default times st column oldpeak htna apo -0,26 ews 6.2 opote eimaste komple

x_noisy.groupby(['Oldpeak']).size()

Oldpeak
-2.504218    1
-2.179169    1
-1.614913    1
-1.007732    1
-0.950449    1
            ..
 4.168354    1
 4.445570    1
 4.914994    1
 5.549065    1
 6.127666    1
Length: 918, dtype: int64

In [ ]:
print(x_noisy)

     Age  sex ChestPainType   RestingBP  Cholesterol FastingBS RestingECG  \
0     40    1           0.5  140.028725   289.047493       0.0          0   
1     49    0             0  159.951914   179.976347       0.0          0   
2     37    1          0.75  129.886267   283.062885       0.0       0.75   
3     48    0           0.5  137.890466   214.003089       0.0          0   
4     54    1             0  149.997806   194.841817       0.0          0   
..   ...  ...           ...         ...          ...       ...        ...   
913   45    1             1  109.982578   264.061031       0.0          0   
914   68    1             0  144.178860   193.091814       1.0          0   
915   56    1           0.5  129.978099   131.002007       0.0          0   
916   57    0          0.75  129.952755   235.816610       0.0       0.75   
917   38    0             0  137.992164   174.968208       0.0          0   

          MaxHR ExerciseAngina   Oldpeak ST_Slope  HeartDisease  
0    171.

In [ ]:
######################################  vazw merikes random NaN times ##################################################################################

cols = ['Age', 'sex', 'ChestPainType', 'Cholesterol', 'ExerciseAngina']                  

# frac=0.1 dld st 10% kathe stilis peripou ligo mikrotero apo 10% st synolo tou dataset
for cols in x_noisy[cols]:
    x_noisy.loc[x_noisy.sample(frac=0.1).index, cols] = pd.np.nan

In [ ]:
##################### gia kaliterh katanomh se merikes stiles vazw 10% NaN times kai se merikes stiles 7% NaN times

cols2 = ['RestingBP', 'FastingBS', 'RestingECG', 'MaxHR', 'Oldpeak', 'ST_Slope']                  

for cols2 in x_noisy[cols2]:
    x_noisy.loc[x_noisy.sample(frac=0.07).index, cols2] = pd.np.nan

In [ ]:
count_nan = len(x_noisy) - x_noisy.count()
count_nan


Age               92
sex               92
ChestPainType     92
RestingBP         64
Cholesterol       92
FastingBS         64
RestingECG        64
MaxHR             64
ExerciseAngina    92
Oldpeak           64
ST_Slope          64
HeartDisease       0
dtype: int64

In [ ]:
print(x_noisy)

      Age  sex ChestPainType   RestingBP  Cholesterol FastingBS RestingECG  \
0    40.0  1.0           0.5  140.028725   289.047493       0.0          0   
1     NaN  0.0             0  159.951914   179.976347       0.0          0   
2    37.0  1.0          0.75  129.886267   283.062885       NaN       0.75   
3    48.0  0.0           0.5  137.890466   214.003089       0.0          0   
4    54.0  1.0             0  149.997806   194.841817       NaN          0   
..    ...  ...           ...         ...          ...       ...        ...   
913  45.0  1.0             1  109.982578   264.061031       NaN          0   
914   NaN  1.0           NaN  144.178860   193.091814       1.0        NaN   
915  56.0  NaN           0.5  129.978099   131.002007       0.0          0   
916   NaN  0.0          0.75  129.952755   235.816610       0.0       0.75   
917  38.0  NaN             0  137.992164   174.968208       0.0          0   

          MaxHR ExerciseAngina   Oldpeak ST_Slope  HeartDisease

In [ ]:
x_noisy.dtypes

Age                float64
sex                float64
ChestPainType     category
RestingBP          float64
Cholesterol        float64
FastingBS         category
RestingECG        category
MaxHR              float64
ExerciseAngina    category
Oldpeak            float64
ST_Slope          category
HeartDisease         int64
dtype: object

In [ ]:
######## to teliko dataframe me noise kai outliers kai NaN times ###########################

In [ ]:
print(x_noisy)

      Age  sex ChestPainType   RestingBP  Cholesterol FastingBS RestingECG  \
0    40.0  1.0           0.5  140.028725   289.047493       0.0          0   
1     NaN  0.0             0  159.951914   179.976347       0.0          0   
2    37.0  1.0          0.75  129.886267   283.062885       NaN       0.75   
3    48.0  0.0           0.5  137.890466   214.003089       0.0          0   
4    54.0  1.0             0  149.997806   194.841817       NaN          0   
..    ...  ...           ...         ...          ...       ...        ...   
913  45.0  1.0             1  109.982578   264.061031       NaN          0   
914   NaN  1.0           NaN  144.178860   193.091814       1.0        NaN   
915  56.0  NaN           0.5  129.978099   131.002007       0.0          0   
916   NaN  0.0          0.75  129.952755   235.816610       0.0       0.75   
917  38.0  NaN             0  137.992164   174.968208       0.0          0   

          MaxHR ExerciseAngina   Oldpeak ST_Slope  HeartDisease

In [ ]:
###### type conversion #######

In [ ]:
## tha ta kanw numeric times ola gia na mporesw na doulepsw me tous ouliers

In [ ]:
x_noisy = x_noisy.apply(pd.to_numeric, errors='ignore')

In [ ]:
print(x_noisy)

      Age  sex  ChestPainType   RestingBP  Cholesterol  FastingBS  RestingECG  \
0    40.0  1.0           0.50  140.028725   289.047493        0.0        0.00   
1     NaN  0.0           0.00  159.951914   179.976347        0.0        0.00   
2    37.0  1.0           0.75  129.886267   283.062885        NaN        0.75   
3    48.0  0.0           0.50  137.890466   214.003089        0.0        0.00   
4    54.0  1.0           0.00  149.997806   194.841817        NaN        0.00   
..    ...  ...            ...         ...          ...        ...         ...   
913  45.0  1.0           1.00  109.982578   264.061031        NaN        0.00   
914   NaN  1.0            NaN  144.178860   193.091814        1.0         NaN   
915  56.0  NaN           0.50  129.978099   131.002007        0.0        0.00   
916   NaN  0.0           0.75  129.952755   235.816610        0.0        0.75   
917  38.0  NaN           0.00  137.992164   174.968208        0.0        0.00   

          MaxHR  ExerciseAn

In [ ]:
###### final 
## data types 
x_noisy.to_csv('out.csv')


### to kanw upload st github --> https://github.com/gkolovos/kaggleDATAset

In [ ]:
x_noisy.dtypes

Age               float64
sex               float64
ChestPainType     float64
RestingBP         float64
Cholesterol       float64
FastingBS         float64
RestingECG        float64
MaxHR             float64
ExerciseAngina    float64
Oldpeak           float64
ST_Slope          float64
HeartDisease        int64
dtype: object

In [ ]:
print(x_noisy)

      Age  sex  ChestPainType   RestingBP  Cholesterol  FastingBS  RestingECG  \
0    40.0  1.0           0.50  140.028725   289.047493        0.0        0.00   
1     NaN  0.0           0.00  159.951914   179.976347        0.0        0.00   
2    37.0  1.0           0.75  129.886267   283.062885        NaN        0.75   
3    48.0  0.0           0.50  137.890466   214.003089        0.0        0.00   
4    54.0  1.0           0.00  149.997806   194.841817        NaN        0.00   
..    ...  ...            ...         ...          ...        ...         ...   
913  45.0  1.0           1.00  109.982578   264.061031        NaN        0.00   
914   NaN  1.0            NaN  144.178860   193.091814        1.0         NaN   
915  56.0  NaN           0.50  129.978099   131.002007        0.0        0.00   
916   NaN  0.0           0.75  129.952755   235.816610        0.0        0.75   
917  38.0  NaN           0.00  137.992164   174.968208        0.0        0.00   

          MaxHR  ExerciseAn